In [1]:
# n_aug = 0,1,2,4,8,16,32
# N = 500,2000,5000,Full
# 


In [2]:
from methods import *
import os, shutil

In [3]:
#user inputs

#load hyperparameters
sizes = ['1_tiny', '2_small', '3_standard', '4_full']
size_folders = ['naug_sized_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

if not os.path.isdir('size_data_f1'):
    os.mkdir('size_data_f1')
#number of words for input
input_size_list = [50, 50, 40, 25, 25]

alphas = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300

In [4]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=20, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [5]:
#for each method
a_method = 'weda' #### TOGGLE EDA/WEDA
tfidf_func = 0
n_augs = [0]

for n_aug in n_augs:

    writer_cnn = open('outputs_naug/cnn_' + a_method + '_' +'tfidf'+str(tfidf_func)+'_'+'naug'+str(n_aug) +'_'+ get_now_str() + '.txt', 'w')

    #for storing the performances
    performances_cnn = {size_folder:[] for size_folder in size_folders}

    #for each percentage dataset
    for size_folder in size_folders:

        writer_cnn.write(size_folder + '\n')

        #get all six datasets
        dataset_folders = [size_folder + '/' + s for s in datasets]



        #for each dataset
        for i in range(len(dataset_folders)):

            #initialize all the variables
            dataset_folder = dataset_folders[i]
            dataset = datasets[i]
            num_classes = num_classes_list[i]
            input_size = input_size_list[i]
            word2vec_pickle = dataset + '/word2vec.p'
            word2vec = load_pickle(word2vec_pickle)



            train_path = dataset_folder + '/train_orig.txt'#'/train_weda_naug_'+str(n_aug)+'.txt' #### TOGGLE EDA/WEDA
            test_path = dataset + '/test.txt'
            acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
            print("cnn aug ", acc, dataset_folder)

            performances_cnn[size_folder].append(acc)
            """
            acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
            performances_rnn[alpha].append(acc)
            print("rnn aug", acc)
            """

    writer_cnn.write(str(performances_cnn) + '\n')
    #writer_rnn.write(str(performances_rnn)+"\n")
    for size_folder in performances_cnn:
        line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
        writer_cnn.write(line + '\n')
        print(line)
    """
    for alpha in performances_rnn:
        line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
        writer_rnn.write(line + '\n')
        print(line)
    """
    print(performances_cnn)

    writer_cnn.close()    


Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 0.6885 - accuracy: 0.5267 - val_loss: 0.6864 - val_accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 108ms/step - loss: 0.6685 - accuracy: 0.5444 - val_loss: 0.6785 - val_accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 114ms/step - loss: 0.6473 - accuracy: 0.5644 - val_loss: 0.6673 - val_accuracy: 0.5800
Epoch 4/20
1/1 [==============================] - 0s 119ms/step - loss: 0.6236 - accuracy: 0.6378 - val_loss: 0.6530 - val_accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 115ms/step - loss: 0.6020 - accuracy: 0.7911 - val_loss: 0.6403 - val_accuracy: 0.6200
Epoch 6/20
1/1 [==============================] - 0s 115ms/step - loss: 0.5796 - accuracy: 0.8556 - val_loss: 0.6293 - val_accuracy: 0.6200
Epoch 7/20
1/1 [==============================] - 0s 111ms/step - loss: 0.5560 - accuracy: 0.8778 - val_loss: 0.6190 - val_accuracy: 0.6400
Epoch 8/20
1/1 [=======

7/7 [==============================] - 2s 196ms/step - loss: 0.6842 - accuracy: 0.5328 - val_loss: 0.6407 - val_accuracy: 0.7581
Epoch 2/20
7/7 [==============================] - 1s 159ms/step - loss: 0.6191 - accuracy: 0.7838 - val_loss: 0.5698 - val_accuracy: 0.7730
Epoch 3/20
7/7 [==============================] - 1s 159ms/step - loss: 0.5435 - accuracy: 0.7778 - val_loss: 0.4932 - val_accuracy: 0.7865
Epoch 4/20
7/7 [==============================] - 1s 165ms/step - loss: 0.4555 - accuracy: 0.8080 - val_loss: 0.4411 - val_accuracy: 0.7851
Epoch 5/20
7/7 [==============================] - 1s 155ms/step - loss: 0.3927 - accuracy: 0.8283 - val_loss: 0.4251 - val_accuracy: 0.7946
Epoch 6/20
7/7 [==============================] - 1s 151ms/step - loss: 0.3533 - accuracy: 0.8490 - val_loss: 0.4193 - val_accuracy: 0.8041
Epoch 7/20
7/7 [==============================] - 1s 154ms/step - loss: 0.3098 - accuracy: 0.8720 - val_loss: 0.4143 - val_accuracy: 0.8108
Epoch 8/20
7/7 [===============